# SeeK-path: the k-path finder and visualizer

For details see http://materialscloud.org/tools/seekpath/input_structure/

In [ ]:
# Credit: https://github.com/giovannipizzi/seekpath/blob/develop/webservice/static/js/orig/BZVisualizer.js
import numpy as np
import pythreejs as THREE
from seekpath.brillouinzone import brillouinzone

class BZVisualizer(THREE.Renderer):
    def __init__(self, res, size=300):
        b1, b2, b3 = np.array(seekpath.hpkot.tools.get_reciprocal_cell_rows(res['primitive_lattice']))
        faces_data = brillouinzone.get_BZ(b1=b1, b2=b2, b3=b3)
    
        # construct WebGL scene
        scene = []
    
        # brillouinzone
        #bz_mat = THREE.BasicMaterial(color="#d53e4f", opacity=0.5, transparent=True, side='FrontSide')
        bz_mat = THREE.BasicMaterial(color="#d53e4f", wireframe=True)
        bz_geom = THREE.PlainGeometry(vertices=faces_data['triangles_vertices'], faces=faces_data['triangles'])
        bz_mesh = THREE.Mesh(geometry=bz_geom, material=bz_mat);
        scene.append(bz_mesh)
    
        # kpoints
        point_mat = THREE.BasicMaterial(color="#3288bd", opacity=1.0, transparent=False)
        for k,v in res['point_coords'].items():
            pos = v[0] * b1 + v[1] * b2 + v[2] * b3
            point = THREE.Mesh(geometry=THREE.SphereGeometry(radius=0.05), material=point_mat, position=pos.tolist())
            scene.append(point)
 
        # paths
        line_mat = THREE.LineBasicMaterial(linewidth=4, color='#045add')
        for p in res['path']:
            vertices = []
            for i in p:
                v = res['point_coords'][i]
                pos = v[0] * b1 + v[1] * b2 + v[2] * b3
                vertices.append(pos.tolist())
                line_geo = THREE.PlainGeometry(vertices=vertices)
            path = THREE.Line(geometry=line_geo, material=line_mat, type='LineStrip')
            scene.append(path)
    
        # setup renderer
        cam = THREE.PerspectiveCamera(position=[5.0, 0, 0])
        ctrls = [THREE.OrbitControls(controlling=cam)]
        super(THREE.Renderer, self).__init__(camera=cam, scene=THREE.Scene(children=scene), controls=ctrls, width="%d"%size, height="%d"%size)

In [ ]:
#Credit: https://github.com/giovannipizzi/seekpath/blob/develop/webservice/seekpath_app.py
import ase.io
import seekpath, seekpath.hpkot
from glob import glob
import ipywidgets as widgets
from IPython.display import display
import nglview as nv

structs = widgets.Dropdown(options=glob("./structures/*.xsf"), description='Structure:')

def get_visualizer(fn):
    atoms = ase.io.read(fn)
    system = (atoms.cell, atoms.get_scaled_positions(), atoms.get_atomic_numbers())
    res = seekpath.hpkot.get_path(system, with_time_reversal=False) 
    bz = BZVisualizer(res)
    w = nv.NGLWidget(nv.ASEStructure(atoms))
    w.add_ball_and_stick()
    w.center_view()
    return widgets.HBox((bz, w,))

app = widgets.VBox((structs, get_visualizer(structs.value)))

def on_change(change):
    app.children[1].close()
    app.children = (structs, get_visualizer(structs.value))
    
structs.observe(on_change, names='value') 

display(app)